### Import needed libraries

In [ ]:
# library provided by EPOCH for reading .sdf output files into Python
import sdf_helper as sh

# python plotting library similar to MATLAB(TM)
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# signal processing, needed for convolution
from scipy import signal

# Python array manipulation
import numpy as np

# various math functions 
import math as m

In [ ]:
# show plots in the notebook
%matplotlib inline

### Define useful functions

In [ ]:
from collections import OrderedDict

In [ ]:
def get_sdf_files(path):
    r"""Given a ``path``, it returns an ordered dictionary containing all the .sdf files in that path, 
    in the form {#### : /../../####.sdf}"""
    
    sdfs = glob.glob(os.path.join(path, '*.sdf')) 
    sdf_dict = {int(sdf.split('.')[0][-4:]):sdf for sdf in sdfs}
    sdf_ord_dict = OrderedDict(sorted(sdf_dict.items(), key=lambda t: t[1])) 
    
    return sdf_ord_dict

In [ ]:
def center_linspace(space):
    r"""
    
    Parameters
    ----------
    space : ndarray
        A one-dimensional array, of linearly spaced numbers. 
    
    Returns
    -------
    out : ndarray
        Also an array of the same spacing, this time centered around 0.
    """
    
    assert np.ndim(space) == 1
    size = space.size
    first = space[0]
    last = space[-1]
    length = last - first
    end = length/2
    
    _, step = np.linspace(first, last, size, retstep=True)
    
    out, out_step = np.linspace(-end, end, size, retstep=True) 
    assert m.isclose(out_step, step)
    return out

In [ ]:
def list_sdf_variables(data):
    r"""Lists all the quantities from the .sdf file.
    
    Parameters
    ----------
    data : ``sdf.Blocklist``
        The results of calling sdf.read on an .sdf file.
    """
    dct = data.__dict__
    for key in sorted(dct):
        try:
            val = dct[key]
            print('{} {} {}'.format(key, type(val),
                  np.array2string(np.array(val.dims), separator=', ')))
        except:
            pass

In [ ]:
def colorbar(mappable):
    r"""Constructs a scaled colorbar for a given plot.
    
    Parameters
    ----------
    mappable : The Image, ContourSet, etc. to which the colorbar applies.
    """
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    return fig.colorbar(mappable, cax=cax)

In [ ]:
def gauss_kern_nd(n, sizes):
    r"""Constructs a Gaussian kernel in any number of dimensions.
    
    Parameters
    ----------
    n : int
        The number of dimensions. Must be 1 or above.
    sizes : list
        The sizes along the various dimensions, ie. in 3D this would be [size_x, size_y, size_x]. The length of
        this list should be either 1 or n. If it only contains one element, eg. [size], it is assumed that
        size_x = size_y = size_x = size.
    
    Returns
    -------
    g : ndarray
        The kernel, with dimensions (in 3d) (2*size_x+1, 2*size_y+1, 2*size_z+1).
    """
    assert n > 0, 'at least 1d required'
    no_sizes = len(sizes)
    assert no_sizes == 1 or no_sizes == n, 'either give one size or all of them'
    
    for i in range(n - no_sizes):
        sizes.append(sizes[0])
        
    slices = tuple(slice(-size,size+1,None) for size in sizes)
    
    XXX = np.mgrid[slices]
    
    g = np.ones(XXX.shape[1:], dtype=np.float64)
    
    for X, size in zip(XXX, sizes):
        g = g * np.exp(-X**2/size)
        
    return g / g.sum()

In [ ]:
def smooth(data, n, sizes):
    r"""Smoothens the input by performing a convolution with a Gaussian kernel.
    
    Parameters
    ----------
    data : ndarray
        Input data, with ``n`` dimensions.
    n : int
        The number of dimensions. Must be 1 or above.
    sizes : list
        The sizes along the various dimensions, ie. in 3D this would be [size_x, size_y, size_x]. The length of
        this list should be either 1 or n. If it only contains one element, eg. [size], it is assumed that
        size_x = size_y = size_x = size.
        
    Returns
    -------
    out : ndarray
        The smoothed input, of shape ```data.shape - 2 * sizes```.
    """
    
    g = gauss_kern_nd(n, sizes)
    out = signal.convolve(data, g, mode='valid')
    return(out)

bash console commands can be run inside the notebook, eg.

The following code will download the data needed for this notebook automatically using `curl`. It may take some time (the archive is 1.2 GB), so please wait when the kernel is busy. You will need to set `download_datasets` to `True` before using it.

In [ ]:
download_datasets = False
if download_datasets:
    !curl -sSO https://ndownloader.figshare.com/articles/5545165/versions/1
    print ("Downloaded the EPOCH data from figshare.")
    !unzip 1 
    
    print ("All done!")

In [ ]:
# these are the .sdf files used in the notebook. they must be present in the same folder when the notebook is run
!ls -lsa *.sdf

In [ ]:
# these are the corresponding EPOCH input decks
!ls -lsa *.deck

# 1D case

In [ ]:
!cat 1dinput.deck

For the 1D case, we just use `numpy` and `matplolib` for low-level plotting and data analysis.

In [ ]:
# load the sdf file produced by EPOCH
fname = '1d.sdf'
data_1d = sh.getdata(fname);

In [ ]:
sh.list_variables(data_1d)

In [ ]:
# assign separate variables to the various quantities we want to look at
grid = data_1d.Grid_Grid_mid
nele = data_1d.Derived_Number_Density_ele
ex = data_1d.Electric_Field_Ex
ey = data_1d.Electric_Field_Ey

In [ ]:
sh.plot_auto(nele)

In [ ]:
# the .data attribute contains the raw data we need to plot
(x,) = grid.data
# convert to micrometers
x = x*1e6

rho = nele.data

In [ ]:
# define size of Gaussian kernel used for noise reduction
kern_size = 100
rho_smooth = smooth(rho, 1, [kern_size])

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=[14,3])

axes[0].plot(x, rho)
axes[1].plot(x[kern_size:-kern_size], rho_smooth)

axes[0].set_title('raw data')
axes[1].set_title('after convolution with Gaussian kernel')

for ax in axes:
    # the labels and units were contained in the .sdf file
    ax.set_xlabel(nele.grid_mid.labels[0] + r' $(\mu m)$', labelpad=-1)
    ax.set_ylabel(nele.name + r' $(' + nele.units + r')$');

plt.tight_layout(h_pad=1)

# saves the generated figure to disk in the same folder that the notebook is ran from
fig.savefig('1d_rho.png')

In [ ]:
sh.plot_auto(ex)

In [ ]:
sh.plot_auto(ey)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=[14,3])

for ax, ef in zip(axes, (ex.data, ey.data)):
    ax.plot(x, ef)

for ax, electric in zip(axes, (ex, ey)):
    ax.set_xlabel(electric.grid_mid.labels[0] + r' $(\mu m)$', labelpad=-1)
    ax.set_ylabel(electric.name + r' $(' + electric.units + r')$');

fig.suptitle('components of the electric field')
plt.tight_layout(h_pad=1)